# Latitude, Longitude for any pixel in a GeoTiff File
How to generate the latitude and longitude for a pixel at any given position in a GeoTiff file.

In [1]:
from osgeo import ogr, osr, gdal

In [2]:
# opening the geotiff file
ds = gdal.Open('/Users/macbook/Documents/BTP/Satellite/Data/LC08_L1TP_150041_20171027_20171027_01_RT/LC08_L1TP_150041_20171027_20171027_01_RT_B2.TIF')

In [3]:
col, row, band = ds.RasterXSize, ds.RasterYSize, ds.RasterCount
print(col, row, band)

(7551, 7701, 1)


In [4]:
xoff, a, b, yoff, d, e = ds.GetGeoTransform()
print(xoff, a, b, yoff, d, e)

# details about the params: GDAL affine transform parameters
# xoff,yoff = left corner 
# a,e = weight,height of pixels
# b,d = rotation of the image (zero if image is north up)

(619185.0, 30.0, 0.0, 3151515.0, 0.0, -30.0)


In [5]:
def pixel2coord(x, y):
    """Returns global coordinates from coordinates x,y of the pixel"""
    xp = a * x + b * y + xoff
    yp = d * x + e * y + yoff
    return(xp, yp)

In [6]:
x,y = pixel2coord(col/2,row/2)
print (x, y)

(732435.0, 3036015.0)


#### These global coordinates are in a *projected coordinated system*, which is a representation of the spheroidal earth's surface, but flattened and distorted onto a plane.
#### To convert these into latitude and longitude, we need to convert these coordinates into *geographic coordinate system*.

In [7]:
# get the existing coordinate system
old_cs= osr.SpatialReference()
old_cs.ImportFromWkt(ds.GetProjectionRef())

# create the new coordinate system
wgs84_wkt = """
GEOGCS["WGS 84",
    DATUM["WGS_1984",
        SPHEROID["WGS 84",6378137,298.257223563,
            AUTHORITY["EPSG","7030"]],
        AUTHORITY["EPSG","6326"]],
    PRIMEM["Greenwich",0,
        AUTHORITY["EPSG","8901"]],
    UNIT["degree",0.01745329251994328,
        AUTHORITY["EPSG","9122"]],
    AUTHORITY["EPSG","4326"]]"""
new_cs = osr.SpatialReference()
new_cs.ImportFromWkt(wgs84_wkt)

# create a transform object to convert between coordinate systems
transform = osr.CoordinateTransformation(old_cs,new_cs) 

In [8]:
# converting into geographic coordinate system
latlong = transform.TransformPoint(x,y)

In [9]:
print (latlong)

(71.351317510464, 27.42778025378802, 0.0)


In [10]:
# rb = ds.GetRasterBand(1)
px,py = col/2,row/2                   # the pixel location
pix = ds.ReadAsArray(px,py,1,1)     
print pix[0]                        # pixel value

[11647]
